In [1]:
import pandas as pd
import os

In [2]:
import sys  
sys.path.insert(0, 'src')
from model import MCSE, BertForCL, RobertaForCL, ResNetVisnModel, MLPLayer

In [3]:
sys.path.insert(0, 'preprocess')
from extract_visn_feature import ResnetFeatureExtractor

In [4]:
refann_df = pd.read_csv('data/ref_data_union.csv', index_col=0)
display(refann_df)

,article,text,sentence,start,end,local_ix,par_ix,ax,related_images,ref_no,tokens,aat_terms,all_terms,term_density,term_density_gen
43,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...","Exeter Cathedral, properly known as the Cathed...",0,199,0,8,43,"['File:Exeter_Cathedral_Nave,_Exeter,_UK_-_Dil...",3,"['exeter', 'cathedral', ',', 'properly', 'know...","['cathedral', 'cathedral', 'church', 'cathedra...","['cathedral', 'cathedral', 'church', 'cathedra...",0.166667,0.166667
44,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...",The present building was complete by about 140...,200,613,0,8,44,"['File:Exeter_Cathedral_Nave,_Exeter,_UK_-_Dil...",5,"['the', 'present', 'building', 'wa', 'complete...","['building', 'set', 'misericords', 'clock', 's...","['building', 'set', 'misericords', 'clock', 's...",0.136986,0.136986
45,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...",A Saxon minster already existing within the to...,615,838,0,8,45,['File:Exeter_Cathedral_9578.jpg'],1,"['a', 'saxon', 'minster', 'already', 'existing...","['minster', 'town', 'seat', 'service', 'door',...","['minster', 'town', 'seat', 'service', 'door',...",0.200000,0.200000
46,Exeter Cathedral,In 1107 William Warelwast was appointed to the...,In 1107 William Warelwast was appointed to the...,0,134,1,9,46,[],0,"['in', '1107', 'william', 'warelwast', 'wa', '...","['catalyst', 'building', 'cathedral']","['catalyst', 'building', 'cathedral']",0.111111,0.111111
47,Exeter Cathedral,In 1107 William Warelwast was appointed to the...,"Its official foundation was in 1133, during Wa...",136,238,1,9,47,[],0,"['it', 'official', 'foundation', 'wa', 'in', '...",['foundation'],['foundation'],0.047619,0.047619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10433,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,This was due to a monetary disagreement that l...,866,1050,2,1899,10433,[],0,"['this', 'wa', 'due', 'to', 'a', 'monetary', '...","['led', 'statue', 'home']","['led', 'statue', 'home']",0.090909,0.090909
10434,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was a memorial to the Mouscron p...,1051,1329,2,1899,10434,[],0,"['the', 'sculpture', 'wa', 'a', 'memorial', 't...","['sculpture', 'memorial', 'tabernacle', 'sculp...","['sculpture', 'memorial', 'parent', 'tabernacl...",0.113208,0.132075
10435,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,"While Michelangelo was alive, the Madonna was ...",1330,1420,2,1899,10435,[],0,"['while', 'michelangelo', 'wa', 'alive', ',', ...",['sculpture'],['sculpture'],0.055556,0.055556
10436,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was twice recovered after being ...,1421,1551,2,1899,10436,[],0,"['the', 'sculpture', 'wa', 'twice', 'recovered...",['sculpture'],['sculpture'],0.050000,0.050000


In [5]:
pic_df = pd.read_csv('data/image_data.csv')

In [6]:
display(pic_df)

,Unnamed: 0,article,wiki_url,caption,filename
0,3,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,The cathedral in 1830,Exeter_Cathedral_NW_view_W_Deeble_after_R_Brow...
1,4,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,"Inside the cathedral, showing the vaulted ceil...",Inside_Exeter_Cathedral.jpg
2,5,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,Detail of the vaulted ceiling,Exeter_Cathedral_nave_vaulted_ceiling.jpg
3,6,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,"One of the misericords, depicting a pipe and t...",Misericordtaborer.png
4,7,Exeter Cathedral,https://upload.wikimedia.org/wikipedia/commons...,The Minstrels' Gallery,Exeter_Cathedral_Minstrels'_Gallery.jpg
...,...,...,...,...,...
1002,1930,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Madonna by Michelangelo, 1501-1504",Michelangelo's_Madonna_and_Child_in_Brugge.jpg
1003,1931,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,Madonna by Michelangelo,Michelangelos_Madonna_OLV-Kerk_Brugge.jpg
1004,1932,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Crucifixion by Anthony van Dyck, 1626",Antoon_van_Dijck.Kruisiging.JPG
1005,1933,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Supper at Emmaüs, formerly ascribed to Caravaggio",Caravaggio_Liebfrauenkirche_Abendmahl_in_Emmau...


In [36]:
all_sents_df = pd.read_csv('data/wsr_all_sents.csv')
display(all_sents_df)

,Unnamed: 0,article,text,sentence,start,end,position,par_id,sent_id,article_id
0,0,Notre-Dame Basilica (Montreal),Notre-Dame Basilica (French: Basilique Notre-D...,Notre-Dame Basilica (French: Basilique Notre-D...,0,147,0,0,0,0
1,1,Notre-Dame Basilica (Montreal),Notre-Dame Basilica (French: Basilique Notre-D...,The church is located at 110 Notre-Dame Street...,148,239,0,0,1,0
2,2,Notre-Dame Basilica (Montreal),Notre-Dame Basilica (French: Basilique Notre-D...,It is located next to the Saint-Sulpice Semina...,240,324,0,0,2,0
3,3,Notre-Dame Basilica (Montreal),Notre-Dame Basilica (French: Basilique Notre-D...,The interior of the church is amongst the most...,324,454,0,0,3,0
4,4,Notre-Dame Basilica (Montreal),Notre-Dame Basilica (French: Basilique Notre-D...,The vaults are coloured deep blue and decorate...,455,615,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...
11273,11273,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,This was due to a monetary disagreement that l...,866,1050,2,1899,11273,97
11274,11274,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was a memorial to the Mouscron p...,1051,1329,2,1899,11274,97
11275,11275,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,"While Michelangelo was alive, the Madonna was ...",1330,1420,2,1899,11275,97
11276,11276,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was twice recovered after being ...,1421,1551,2,1899,11276,97


# Generated Captions

In [40]:
import pickle

In [7]:
cap_df = pd.read_csv("../../image_data_comp.csv")
#display(cap_df)

In [8]:
cap_df.drop(columns=cap_df.columns[:9], axis=1, inplace=True)

In [9]:
display(cap_df)

,article,wiki_url,caption,filename,article_id,generated_caption,generated_caption_IBM
0,Cathedral of Saint Paul (Minnesota),https://upload.wikimedia.org/wikipedia/commons...,"The Kittson mansion, demolished to make way fo...","Kittson_House_at_St._Paul,_Minnesota.gif",14,the house was built and is now a museum.,a black and white photo of a clock tower .
1,Cathedral of Saint Paul (Minnesota),https://upload.wikimedia.org/wikipedia/commons...,Cathedral of Saint Paul as seen from the Landm...,Cathedral_of_St._Paul_from_the_Landmark_Center...,14,the cathedral from the side.,a tall building with a clock on it
2,Cathedral of Saint Paul (Minnesota),https://upload.wikimedia.org/wikipedia/commons...,Baldachin of the Cathedral of Saint Paul,"Altar_of_Cathedral_of_Saint_Paul_in_St._Paul,_...",14,the altar of the church.,a large clock mounted to the side of a building .
3,Cathedral of Saint Paul (Minnesota),https://upload.wikimedia.org/wikipedia/commons...,"May 15, 2013 – Cathedral of Saint Paul Organ Case",Cathedral_of_Saint_Paul_Organ_Case.jpg,14,the organ in the cathedral.,a large clock mounted to the side of a building .
4,Notre-Dame Basilica (Montreal),https://upload.wikimedia.org/wikipedia/commons...,"The Notre-Dame Church, with its replacement be...",Place_d_Armes_Montreal_1828.jpg,0,file not found,file not found
...,...,...,...,...,...,...,...
1818,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Madonna by Michelangelo, 1501-1504",Michelangelo's_Madonna_and_Child_in_Brugge.jpg,97,the altar in the church.,a large clock mounted to the side of a building .
1819,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,Madonna by Michelangelo,Michelangelos_Madonna_OLV-Kerk_Brugge.jpg,97,a statue of person and builder.,a statue of a man sitting on a bench .
1820,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Crucifixion by Anthony van Dyck, 1626",Antoon_van_Dijck.Kruisiging.JPG,97,file not found,file not found
1821,"Church of Our Lady, Bruges",https://upload.wikimedia.org/wikipedia/commons...,"Supper at Emmaüs, formerly ascribed to Caravaggio",Caravaggio_Liebfrauenkirche_Abendmahl_in_Emmau...,97,"painting artist, oil on canvas.",a group of people sitting around a table eatin...


In [44]:
with open('../sentence_based_mturk.pkl', 'rb') as f:
    sent_df = pickle.load(f)
display(sent_df)

,article,text,sentence,start,end,sent_no,parag_id,in_preselection,Ann-0,Ann-1,Ann-2,captions,filenames,Ann-0-index,Ann-1-index,Ann-2-index,mis_annt,Ann-Maj,Ann-Maj-Cons,filename
48,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...","Exeter Cathedral, properly known as the Cathed...",0,199,0,8,True,"{0: {1: {'miss_ant': 1, 'onset': 0, 'offset': ...","{0: {}, 1: {1: {'miss_ant': 0, 'onset': 0, 'of...","{0: {1: {'miss_ant': 0, 'onset': 66, 'offset':...",[The 17th-century organ case (enlarged in 1891...,"[File:Exeter_Cathedral_9578.jpg, File:Exeter_C...",[0],"[1, 2]","[0, 1]",None,"[0, 1]",[1],NaN
49,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...",The present building was complete by about 140...,200,418,0,8,True,"{0: {1: {'miss_ant': 1, 'onset': 0, 'offset': ...","{0: {}, 1: {1: {'miss_ant': 0, 'onset': 200, '...","{0: {1: {'miss_ant': 0, 'onset': 66, 'offset':...",[The 17th-century organ case (enlarged in 1891...,"[File:Exeter_Cathedral_9578.jpg, File:Exeter_C...","[0, 1, 2]","[1, 2]","[0, 1, 2]",None,"[0, 1, 2]","[1, 2]",NaN
50,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...","The founding of the cathedral at Exeter, dedic...",418,613,0,8,True,"{0: {1: {'miss_ant': 1, 'onset': 0, 'offset': ...","{0: {}, 1: {1: {'miss_ant': 0, 'onset': 200, '...","{0: {1: {'miss_ant': 0, 'onset': 495, 'offset'...",[The 17th-century organ case (enlarged in 1891...,"[File:Exeter_Cathedral_9578.jpg, File:Exeter_C...","[0, 2]",[1],"[0, 2]",None,"[0, 2]",[2],NaN
51,Exeter Cathedral,"Exeter Cathedral, properly known as the Cathed...",A Saxon minster already existing within the t...,614,838,0,8,True,"{0: {1: {'miss_ant': 1, 'onset': 0, 'offset': ...","{0: {}, 1: {}, 2: {}}","{0: {}, 1: {}, 2: {1: {'miss_ant': 0, 'onset':...",[The 17th-century organ case (enlarged in 1891...,"[File:Exeter_Cathedral_9578.jpg, File:Exeter_C...",[0],[],[2],None,[],[],NaN
52,Exeter Cathedral,In 1107 William Warelwast was appointed to the...,In 1107 William Warelwast was appointed to the...,0,134,1,9,True,{0: {}},{0: {}},{0: {}},[The South Tower where the 12 bells hang],"[File:South_tower,_Exeter_Cathedral_-_geograph...",[],[],[],None,[],[],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11273,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,This was due to a monetary disagreement that l...,866,1050,2,1899,True,{},{},{},[],[],[],[],[],None,[],[],NaN
11274,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was a memorial to the Mouscron p...,1051,1329,2,1899,True,{},{},{},[],[],[],[],[],None,[],[],NaN
11275,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,"While Michelangelo was alive, the Madonna was ...",1330,1420,2,1899,True,{},{},{},[],[],[],[],[],None,[],[],NaN
11276,"Church of Our Lady, Bruges",The altarpiece of the large chapel in the sout...,The sculpture was twice recovered after being ...,1421,1551,2,1899,True,{},{},{},[],[],[],[],[],None,[],[],NaN


In [45]:
sents = list(sent_df["sentence"])
len(sents)

3432

## Extracting Image Features

In [29]:
paths = [p for p in list(pic_df["filename"]) if p in os.listdir("medium_images")]

In [8]:
len(paths)

994

In [22]:
extractor = ResnetFeatureExtractor("medium_images", "data/", 32)
extractor.extract_vision_features(dataname="medium_images", img_ids=paths)

  2%|█▋                                                                               | 20/994 [00:00<00:10, 96.72it/s]

Start extracting resnet features...


100%|████████████████████████████████████████████████████████████████████████████████| 994/994 [00:36<00:00, 27.51it/s]

	Save features to data/medium_images.hdf5 with hdf5 dataset 'features'.


## Sentences 

In [10]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data.dataloader import DataLoader
import numpy as np
import tqdm
import h5py

In [11]:
model_path = "result/mix_flickr/0/mcse"

In [12]:
model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
device = torch.device("cuda:0")
model = model.to(device)

In [10]:
sentences = list(refann_df["sentence"])

In [33]:
def extract_sent_features(sents, model, batch_size, cls=False, mlp=False):
    tensor_sents = []
    sent_feats = []
    for i, sent in enumerate(tqdm.tqdm(sents)):
        if sent == 0: 
            sent_feats.extend(torch.zeros(1, 32, 768))
        else:
            tensor_sents.append(sent)
            if len(tensor_sents) == batch_size:
                tensor_sents = tokenizer.batch_encode_plus(tensor_sents,return_tensors='pt',padding="max_length",max_length=32,
                                                               truncation=True)
                for k in tensor_sents:
                    tensor_sents[k] = tensor_sents[k].to(device)
                #sent_input = torch.stack(tensor_sents).cuda()
                #sent_input = torch.squeeze(sent_input)
                with torch.no_grad():
                    sent_output = model(**tensor_sents)
                sent_feats.extend(sent_output.last_hidden_state.detach().cpu())
                tensor_sents = []
    if len(tensor_sents) > 0: 
        tensor_sents = tokenizer.batch_encode_plus(tensor_sents,return_tensors='pt',padding="max_length",max_length=32,
                                                   truncation=True)
        for k in tensor_sents:
            tensor_sents[k] = tensor_sents[k].to(device)
        #sent_input = torch.stack(tensor_sents).cuda()
        #sent_input = torch.squeeze(sent_input)
        with torch.no_grad():
            sent_output = model(**tensor_sents)
        sent_feats.extend(sent_output.last_hidden_state.detach().cpu())
    sent_feats = torch.stack(sent_feats)
    if cls: 
        sent_feats = sent_feats[:, 0]
    if mlp: 
        proj = MLPLayer(768, 256)
        sent_feats = proj(sent_feats)
    return(sent_feats)

                
    

In [14]:
def save_to_hdf5(feats, output_file):
    h5f = h5py.File(output_file, 'w')
    dset = h5f.create_dataset("features", data=feats)
    h5f.close()
    print("features saved to " + output_file)

In [35]:
#sent_feats = extract_sent_features(sentences, model, 32)
cls_feats = extract_sent_features(sentences, model, 32, cls=True)

100%|█████████████████████████████████████████████████████████████████████████████| 5066/5066 [00:50<00:00, 100.29it/s]


In [38]:
save_to_hdf5(cls_feats, "data/cls_feats.hdf5")

features saved to data/cls_feats.hdf5


In [49]:
captions = list(pic_df["caption"])

In [46]:
caption_feats = extract_sent_features(captions, model, 32)
caption_feats.size()

100%|█████████████████████████████████████████████████████████████████████████████| 1007/1007 [00:09<00:00, 101.88it/s]


torch.Size([1007, 32, 768])

In [52]:
save_to_hdf5(caption_feats, "data/caption_feats.hdf5")

features saved to data/caption_feats.hdf5


In [55]:
all_sents = list(all_sents_df["sentence"])
#print(all_sents)

In [56]:
all_sent_feats = extract_sent_features(all_sents, model, 32)

100%|████████████████████████████████████████████████████████████████████████████| 11278/11278 [02:03<00:00, 91.19it/s]


In [57]:
all_sent_feats.size()

torch.Size([11278, 32, 768])

In [58]:
save_to_hdf5(all_sent_feats, "data/all_sent_feats.hdf5")

features saved to data/all_sent_feats.hdf5


## extracting features for captions 

In [15]:
def get_cap_list(cap_list):
    cap_list_clean = []
    for cap in cap_list:
        if cap in ["file not found", "wrong file format", "status error"]:
            cap_list_clean.append(0)
        else:
            cap_list_clean.append(cap)
    return cap_list_clean

In [52]:
wiki_caps = get_cap_list(list(cap_df["caption"]))
clip_caps = get_cap_list(list(cap_df["generated_caption"]))
IBM_caps = list(cap_df["generated_caption_IBM"])

In [18]:
wiki_feats = extract_sent_features(wiki_caps, model, 32)
wiki_feats.size()

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [00:21<00:00, 86.02it/s]


torch.Size([1823, 32, 768])

In [ ]:
clip_feats = extract_sent_features(clip_caps, model, 32)


In [51]:
clip_feats.size()

torch.Size([1823, 32, 768])

In [53]:
IBM_feats = extract_sent_features(IBM_caps, model, 32)

100%|██████████████████████████████████████████████████████████████████████████████| 1823/1823 [00:18<00:00, 97.88it/s]


In [46]:
sent_feats = extract_sent_features(sents, model, 32)

100%|██████████████████████████████████████████████████████████████████████████████| 3432/3432 [00:34<00:00, 98.30it/s]


In [54]:
sent_feats.size()

torch.Size([3432, 32, 768])

In [55]:
save_to_hdf5(wiki_feats, "data/wiki_cap_feats.hdf5")
save_to_hdf5(sent_feats, "data/new_sent_feats.hdf5")
save_to_hdf5(clip_feats, "data/clip_feats.hdf5")
save_to_hdf5(IBM_feats, "data/IBM_feats.hdf5")

features saved to data/wiki_cap_feats.hdf5
features saved to data/new_sent_feats.hdf5
features saved to data/clip_feats.hdf5
features saved to data/IBM_feats.hdf5


### opening the .hdf5 files

In [61]:
hf = h5py.File('data/medium_images.hdf5', 'r')
#data = hf.get('features').value
data = np.array(hf["features"][:])

data.shape

(994, 2048)

In [63]:
visn_model = ResNetVisnModel(2048, 256)
visn_proj = visn_model(torch.from_numpy(data))
visn_proj.size()

torch.Size([994, 256])

In [ ]:
hf = h5py.File('data/sent_feats.hdf5', 'r')
#data = hf.get('features').value
data = np.array(hf["features"][:])

data.shape

## Sonstiges

In [17]:
from PIL import Image

for p in listdir("medium_images"):
    if p.endswith(".png"):
        fp = "medium_images/" + p
        im = Image.open(fp)
        rgb_im = im.convert('RGB')
        rgb_im.save(os.path.splitext(fp)[0] + ".jpg")

In [15]:
import os

conts = [p.lower() for p in os.listdir("medium_images")]
missing = []
for p in paths: 
    if not p.lower() in conts:
        missing.append(p)

In [16]:
print(len(missing))

13
